In [15]:
import glob
from os.path import dirname, basename
import pandas as pd

In [16]:
data_dir = '../data'
results_dir = '../results'
dataset = 'demo'
sf_inpainting_models = [
    'sf_deepfillv1',
    'sf_deepfillv2',
    'sf_kernelfill',
    'sf_pconvunet',
    'sf_regionfill'
]

object_stats_dfs = []
for path in glob.glob(f'{data_dir}/interim/{dataset}/ObjectStats/*/raw_object_stats.csv'):
    object_stats_df = pd.read_csv(path)
    object_stats_df['object_sequence_name'] = basename(dirname(path))
    object_stats_dfs.append(object_stats_df)
object_stats_df = pd.concat(object_stats_dfs)

results_dfs = []
for model in sf_inpainting_models:
    for path in glob.glob(f'{results_dir}/inpainting/{model}/Evaluation/*/results.csv'):
        results_df = pd.read_csv(path)
        results_df['model'] = model
        sequence_name = basename(dirname(path))
        results_df['sequence_name'] = sequence_name
        results_df['object_sequence_name'] = sequence_name.split('_')[1]
        results_df['object_id'] = int(sequence_name.split('_')[2])
        results_dfs.append(results_df)
results_df = pd.concat(results_dfs)

df = pd.merge(object_stats_df, results_df, on=['object_sequence_name', 'object_id', 'frame_id'])
df

,frame_id,object_id,object_to_screen_ratio,object_sequence_name,mean_squared_error,peak_signal_noise_ratio,structural_similarity,model,sequence_name
0,0,1,0.110939,breakdance,360.423803,22.562669,0.906953,sf_deepfillv1,flamingo_breakdance_001
1,0,1,0.110939,breakdance,189.509466,25.354495,0.932064,sf_deepfillv1,tennis_breakdance_001
2,0,1,0.110939,breakdance,268.013954,23.849230,0.905166,sf_deepfillv2,flamingo_breakdance_001
3,0,1,0.110939,breakdance,206.578341,24.979956,0.928127,sf_deepfillv2,tennis_breakdance_001
4,0,1,0.110939,breakdance,214.097310,24.824691,0.920352,sf_kernelfill,flamingo_breakdance_001
...,...,...,...,...,...,...,...,...,...
2180,68,1,0.036728,tennis,41.109530,31.991378,0.978259,sf_kernelfill,flamingo_tennis_001
2181,68,1,0.036728,tennis,163.697439,25.990385,0.966105,sf_pconvunet,breakdance_tennis_001
2182,68,1,0.036728,tennis,178.500987,25.614397,0.963143,sf_pconvunet,flamingo_tennis_001
2183,68,1,0.036728,tennis,54.437164,30.771849,0.975505,sf_regionfill,breakdance_tennis_001


In [17]:
metrics = pd.DataFrame()
metrics['model'] = df['model']
metrics['sequence_name'] = df['sequence_name']
metrics['mean_squared_error'] = df['mean_squared_error']
metrics['peak_signal_noise_ratio'] = df['peak_signal_noise_ratio']
metrics['structural_similarity'] = df['structural_similarity']
metrics.groupby('model').mean()

,mean_squared_error,peak_signal_noise_ratio,structural_similarity
model,,,
sf_deepfillv1,153.933700,27.014518,0.950027
sf_deepfillv2,102.718932,28.358518,0.950726
sf_kernelfill,72.947274,30.247913,0.962614
sf_pconvunet,264.537325,24.274194,0.934383
sf_regionfill,68.226020,30.560551,0.963308


In [18]:
metrics = pd.DataFrame()
metrics['model'] = df['model']
metrics['sequence_name'] = df['sequence_name']
metrics['mean_squared_error'] = df['mean_squared_error'] * df['object_to_screen_ratio']
metrics['peak_signal_noise_ratio'] = df['peak_signal_noise_ratio'] * df['object_to_screen_ratio']
metrics['structural_similarity'] = df['structural_similarity'] * df['object_to_screen_ratio']
metrics.groupby('model').mean()


,mean_squared_error,peak_signal_noise_ratio,structural_similarity
model,,,
sf_deepfillv1,12.294791,1.792648,0.064068
sf_deepfillv2,7.823281,1.894376,0.064190
sf_kernelfill,5.458718,2.029922,0.065075
sf_pconvunet,20.214764,1.614798,0.062826
sf_regionfill,4.992805,2.060402,0.065135
